In [868]:
import pandas as pd
import datetime
import json
import riak
import urllib2
import pytz
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from pandas.io.data import DataReader
import riak
from riak import RiakClient, RiakNode, RiakObject

def getData(tickerFile, dataSource, start, end):

    rc = RiakClient(pb_port=8087, protocol='pbc')
    added = []
    notAdded = []
    stock = pd.read_csv(tickerFile,sep='\t',header=None)

    #loop over all stock tickers
    for i in range(0,len(stock.head(100))):
        #get daily data for each ticker
        gtemp = pd.DataFrame()
        bucket = rc.bucket('stocks')
        try:
            gtemp = DataReader(stock.ix[i,0],  dataSource, start, end)
        except:
            pass
        
        #didnt get any data
        if len(gtemp) == 0:
            notAdded.append(stock.ix[i,0])
        #got data
        else:
            print(str(i) + ': ' + str(stock.ix[i,0]) + ' : ' + str(float(i)/len(stock)))
            added.append(stock.ix[i,0])
            
            for j in range(0,len(gtemp.index)):
            
                #upload json to Riak Bucket
                date = gtemp.index[j].date()
                riakKey = str(stock.ix[i,0]) + '_' + str(date)
                riakVal = {'OPEN': gtemp.values[j,0], \
                           'HIGH': gtemp.values[j,1], \
                           'LOW': gtemp.values[j,2], \
                           'CLOSE': gtemp.values[j,3], \
                           'VOLUME': gtemp.values[j,4],\
                           'DATE': str(date),\
                           'TICKER': str(stock.ix[i,0])}
                
                obj = RiakObject(rc, bucket, riakKey)
                
                obj.add_index("ticker_bin", str(stock.ix[i,0]))
                obj.add_index("year_int", int(date.year))
                obj.add_index("month_int", int(date.month))
                obj.add_index("day_int", int(date.day))
                
                obj.content_type = 'text/json'
                #obj.data = riakVal
                obj.data = json.dumps(riakVal)
                obj.store()

    return added, notAdded

In [869]:
a,b = getData('NYSE.txt','google',datetime(2010,5,8),datetime(2015,6,8))

0: A : 0.0
1: AA : 0.000304506699147
2: AA-B : 0.000609013398295
3: AAC : 0.000913520097442
4: AAN : 0.00121802679659
5: AAP : 0.00152253349574
6: AAT : 0.00182704019488
7: AAV : 0.00213154689403
8: AB : 0.00243605359318
9: ABB : 0.00274056029233
10: ABBV : 0.00304506699147
11: ABC : 0.00334957369062
12: ABEV : 0.00365408038977
13: ABG : 0.00395858708892
14: ABM : 0.00426309378806
15: ABR : 0.00456760048721
16: ABR-A : 0.00487210718636
17: ABR-B : 0.00517661388551
18: ABR-C : 0.00548112058465
20: ABT : 0.00609013398295
21: ABX : 0.0063946406821
22: ACC : 0.00669914738124
23: ACCO : 0.00700365408039
24: ACE : 0.00730816077954
25: ACG : 0.00761266747868
26: ACH : 0.00791717417783
27: ACI : 0.00822168087698
28: ACM : 0.00852618757613
29: ACN : 0.00883069427527
30: ACP : 0.00913520097442
31: ACRE : 0.00943970767357
34: ACV : 0.010353227771
35: ACW : 0.0106577344702
36: ADC : 0.0109622411693
37: ADM : 0.0112667478685
38: ADPT : 0.0115712545676
39: ADS : 0.0118757612667
40: ADT : 0.012180267

In [11]:
from riak import RiakClient, RiakNode, RiakObject

def riakSearchData(searchBucket, searchTerm, searchVal1, searchVal2):
    myData = {}
    myBucket = RiakClient(pb_port=8087, protocol='pbc').bucket(searchBucket)
    if searchVal2 != None:
        for key in myBucket.get_index(searchTerm, searchVal1, searchVal2): # get all from 2002 to 2012
            myData[key] = json.loads(myBucket.get(key).data)
    else:
        for key in myBucket.get_index(searchTerm, searchVal1): # get all from 2002 to 2012
            myData[key] = json.loads(myBucket.get(key).data)
    return myData

In [12]:
import riak

def storeKV(myBucket, myKey, myVal):
    riak.RiakClient(pb_port=8087, protocol='pbc').bucket(myBucket).new(myKey, data = myVal).store()
    return

In [13]:
import riak

def deleteKey(delBucket, delKey):
    riak.RiakClient(pb_port=8087, protocol='pbc').bucket(delBucket).delete(delKey)
    if riak.RiakClient(pb_port=8087, protocol='pbc').bucket(delBucket).get(delKey).data == None:
        print 'Successful delete: %s' % delKey
    else:
        print 'Failed delete: %s' % delKey
    return

In [14]:
import riak

def deleteAllKeys(delBucket):

    for keys in  riak.RiakClient(pb_port=8087, protocol='pbc').bucket(delBucket).stream_keys():
        for delKey in keys:
            deleteKey(delBucket, delKey)

In [15]:
import riak

def getAllKV(myBucket):
    myData = {}
    riak_bucket = riak.RiakClient(pb_port=8087, protocol='pbc').bucket(myBucket)
    for keys in riak_bucket.stream_keys():
        for key in keys:
            tempData = riak_bucket.get(key).data
            print('Key: %s Value: %s' % (key, tempData))
            myData[key] = tempData
    return myData

In [16]:
import riak

def getValue(myBucket, myKey):
    myVal = json.loads(riak.RiakClient(pb_port=8087, protocol='pbc').bucket(myBucket).get(myKey).data)
    return myVal

In [1]:
import sys
from operator import add

from pyspark import SparkContext



In [3]:
lines = sc.parallelize(['Its fun to have fun,','but you have to know how.']) 
wordcounts = lines.map( lambda x: x.replace(',',' ').replace('.',' ').replace('-',' ').lower()) \
        .flatMap(lambda x: x.split()) \
        .map(lambda x: (x, 1)) \
        .reduceByKey(lambda x,y:x+y) \
        .map(lambda x:(x[1],x[0])) \
        .sortByKey(False) 
wordcounts.take(10)

[(2, 'fun'),
 (2, 'to'),
 (2, 'have'),
 (1, 'how'),
 (1, 'you'),
 (1, 'its'),
 (1, 'know'),
 (1, 'but')]

In [4]:
data = range(0,10000)
distData = sc.parallelize(data)
distData.map(lambda s: s).reduce(lambda a, b: a + b)

49995000

In [177]:
w = json.dumps(riakSearchData('stocks', 'month_int', 6, None))
w

'{"AAP_2014-06-24": {"HIGH": 133.92, "OPEN": 130.67, "VOLUME": 1509069.0, "LOW": 130.67, "DATE": "2014-06-24", "CLOSE": 131.66, "TICKER": "AAP"}, "AAP_2014-06-25": {"HIGH": 131.53, "OPEN": 130.98, "VOLUME": 1124861.0, "LOW": 128.02, "DATE": "2014-06-25", "CLOSE": 129.85, "TICKER": "AAP"}, "AAP_2014-06-26": {"HIGH": 131.71, "OPEN": 130.3, "VOLUME": 1157738.0, "LOW": 130.02, "DATE": "2014-06-26", "CLOSE": 130.53, "TICKER": "AAP"}, "AB_2010-06-25": {"HIGH": 28.37, "OPEN": 28.21, "VOLUME": 316218.0, "LOW": 27.5, "DATE": "2010-06-25", "CLOSE": 27.99, "TICKER": "AB"}, "AAP_2014-06-20": {"HIGH": 129.91, "OPEN": 129.4, "VOLUME": 829234.0, "LOW": 127.89, "DATE": "2014-06-20", "CLOSE": 129.8, "TICKER": "AAP"}, "AA_2011-06-21": {"HIGH": 15.42, "OPEN": 14.94, "VOLUME": 18326596.0, "LOW": 14.92, "DATE": "2011-06-21", "CLOSE": 15.37, "TICKER": "AA"}, "AAT_2013-06-12": {"HIGH": 32.44, "OPEN": 32.17, "VOLUME": 184337.0, "LOW": 31.37, "DATE": "2013-06-12", "CLOSE": 31.38, "TICKER": "AAT"}, "A_2011-06-1

In [31]:
distData = sc.parallelize(w.values())
distData.map(lambda s: s["CLOSE"]).reduce(lambda a, b: (a + b)) / len(w.values())

11.472765957446809

In [26]:
tot = 0
for val in w.values():
    tot = tot + val['CLOSE']
tot = tot/len(w.values())
print tot

11.4727659574


In [97]:
distData = sc.parallelize(a)
t = distData.map(lambda s: riakSearchData(s, 'month_int', 6, None)).collect()

In [205]:
initData = sc.parallelize(riakSearchData('stocks', 'year_int', 2011, 2015))

In [181]:
myData = riakSearchData('stocks', 'year_int', 2011, 2015)

In [202]:
myData.values()

list

In [870]:
data = riakSearchData('stocks', 'year_int', 2011,2015)

In [250]:
data.values()

[{u'CLOSE': 148.05,
  u'DATE': u'2015-04-09',
  u'HIGH': 149.23,
  u'LOW': 146.99,
  u'OPEN': 148.31,
  u'TICKER': u'AAP',
  u'VOLUME': 646873.0},
 {u'CLOSE': 54.05,
  u'DATE': u'2014-10-24',
  u'HIGH': 54.11,
  u'LOW': 53.51,
  u'OPEN': 53.7,
  u'TICKER': u'A',
  u'VOLUME': 1508194.0},
 {u'CLOSE': 53.99,
  u'DATE': u'2014-10-27',
  u'HIGH': 54.11,
  u'LOW': 53.6,
  u'OPEN': 53.94,
  u'TICKER': u'A',
  u'VOLUME': 1105293.0},
 {u'CLOSE': 52.35,
  u'DATE': u'2014-10-20',
  u'HIGH': 52.38,
  u'LOW': 50.86,
  u'OPEN': 52.07,
  u'TICKER': u'A',
  u'VOLUME': 4942206.0},
 {u'CLOSE': 54.0,
  u'DATE': u'2014-10-21',
  u'HIGH': 54.03,
  u'LOW': 52.86,
  u'OPEN': 52.86,
  u'TICKER': u'A',
  u'VOLUME': 2231876.0},
 {u'CLOSE': 82.49,
  u'DATE': u'2013-07-31',
  u'HIGH': 83.07,
  u'LOW': 82.01,
  u'OPEN': 82.25,
  u'TICKER': u'AAP',
  u'VOLUME': 521429.0},
 {u'CLOSE': 82.19,
  u'DATE': u'2013-07-30',
  u'HIGH': 82.8,
  u'LOW': 82.15,
  u'OPEN': 82.53,
  u'TICKER': u'AAP',
  u'VOLUME': 482674.0},
 {u

In [871]:
ar = []

for value in data.values():
    ar.append([value['TICKER'], value['CLOSE'], value['VOLUME'], value['DATE']])

In [872]:
d = sc.parallelize(ar).map(lambda s: (str(s[0]),[s[1],s[2], str(s[3])])).groupByKey().map(lambda x : (x[0], list(x[1])))

In [783]:
d = sc.parallelize(ar).map(lambda s: (str(s[0]),[s[1],s[2], str(s[3])])).groupByKey()

In [928]:
#Take a tuple of tuples in and return something
import numpy

def pairAnalysis(pairTuple):
    
    stockA = pairTuple[0]
    stockAData = list(stockA[1])
    stockAData.sort(key = lambda x: x[2])
    
    stockADates = [x[2] for x in stockAData]
    stockAClose = [x[0] for x in stockAData]
    stockAVolume = [x[1] for x in stockAData]
    
    stockB = pairTuple[1]
    stockBData = list(stockB[1])
    stockBData.sort(key = lambda x: x[2])
    
    stockBDates = [x[2] for x in stockBData]
    stockBClose = [x[0] for x in stockBData]
    stockBVolume = [x[1] for x in stockBData]
    
    start = len(stockAClose)-1-140
    end = len(stockAClose)-1
    
    if stockADates[start:end] != stockBDates[start:end]:
        return 1#str(stockADates[len(stockADates)-1-140])+':'+str(stockBDates[len(stockBDates)-1-140])
    else:
        start = len(stockAClose)-1-140
        end = len(stockAClose)-1
        coint = eg_test(stockAClose[start:end],stockBClose[start:end])
        #print coint
        if (coint[0] != 1):
            return 2
        else:
            signal = [a - coint[1][1]*b - coint[1][0] for a in stockAClose[start:end] for b in stockBClose[start:end]]
            sigMean = numpy.mean(signal)
            sigStd = numpy.std(signal)
            zscore = (signal[len(signal)-1] - sigMean)/sigStd
            if abs(zscore) > 2:
                return [stockA[0], stockAClose[end], stockB[0], stockBClose[end], zscore, coint[1][1], sigMean, sigStd, stockADates[end]]
    return 3

In [929]:
pairs = d.cartesian(d).map(pairAnalysis).collect()

In [930]:
pairs

[2,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 3,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 3,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 2,


In [864]:
def writePairs(pairList):
    
    tradeable = [x for x in pairList if type(x) is list]
    
    rc = RiakClient(pb_port=8087, protocol='pbc')
    bucket = rc.bucket('tradeEntries')
    for pair in tradeable:
        
        key = str(str(pair[0])+ '_' + str(pair[2]))
        val = {'StockA': pair[0], \
                   'CloseA': pair[1], \
                   'StockB': pair[2], \
                   'CloseB': pair[3], \
                   'ZScore': pair[4],\
                   'Beta': pair[5],\
                   'SignalMean': pair[6],\
                   'SignalSD': pair[7],\
                   'Date': pair[8]}
        myDate = pair[8].split('-')
        obj = RiakObject(rc, bucket, key)
        obj.add_index("stocka_bin", str(pair[0]))
        obj.add_index("stockb_bin", str(pair[3]))
        obj.add_index("year_int", int(myDate[0]))
        obj.add_index("month_int", int(myDate[1]))
        obj.add_index("day_int", int(myDate[2]))
        obj.content_type = 'text/json'
        obj.data = val
        obj.data = json.dumps(val)
        obj.store()
        
    return tradeable
    

In [896]:
a = writePairs(pairs)

In [899]:
a

[['AEH',
  25.24,
  'AAN',
  34.53,
  -2.0702104024750518,
  0.013486756468064021,
  1.6091819235144714e-14,
  0.18573101198626543,
  '2015-06-08'],
 ['AEH',
  25.24,
  'AER',
  47.95,
  -2.0967590322665308,
  0.015937876432387962,
  5.7031416626311214e-14,
  0.18648612588070088,
  '2015-06-08'],
 ['AEH',
  25.24,
  'AF',
  13.71,
  -2.1435425554966918,
  0.16969216964417733,
  -1.3162310747500468e-15,
  0.18756504775176494,
  '2015-06-08'],
 ['AF',
  13.71,
  'AEH',
  25.24,
  2.0039385953726558,
  0.54184640314727628,
  2.3092885628430951e-14,
  0.33516534536008191,
  '2015-06-08']]

In [902]:
trades = getAllKV('tradeEntries')

Key: AEH_AER Value: {"CloseB": 47.95, "ZScore": -2.0967590322665308, "CloseA": 25.24, "SignalMean": 5.7031416626311214e-14, "SignalSD": 0.18648612588070088, "Date": "2015-06-08", "StockA": "AEH", "StockB": "AER", "Beta": 0.015937876432387962}
Key: AEH_AF Value: {"CloseB": 13.71, "ZScore": -2.1435425554966918, "CloseA": 25.24, "SignalMean": -1.3162310747500468e-15, "SignalSD": 0.18756504775176494, "Date": "2015-06-08", "StockA": "AEH", "StockB": "AF", "Beta": 0.16969216964417733}
Key: AEH_AAN Value: {"CloseB": 34.53, "ZScore": -2.0702104024750518, "CloseA": 25.24, "SignalMean": 1.6091819235144714e-14, "SignalSD": 0.18573101198626543, "Date": "2015-06-08", "StockA": "AEH", "StockB": "AAN", "Beta": 0.013486756468064021}
Key: AF_AEH Value: {"CloseB": 25.24, "ZScore": 2.0039385953726558, "CloseA": 13.71, "SignalMean": 2.3092885628430951e-14, "SignalSD": 0.33516534536008191, "Date": "2015-06-08", "StockA": "AF", "StockB": "AEH", "Beta": 0.54184640314727628}


In [895]:
deleteAllKeys('tradeEntries')

Successful delete: AF_AGM
Successful delete: AFC_ACC
Successful delete: AES-C_AGX
Successful delete: AEH_AHC
Successful delete: AGO-B_ACN
Successful delete: AF_ACG
Successful delete: AFG_AGO-B
Successful delete: AFC_ACG
Successful delete: AEH_ADS
Successful delete: AED_AHL
Successful delete: AED_ABT
Successful delete: AGO-B_ABG
Successful delete: AF_AFC
Successful delete: AF_AEM
Successful delete: AF_ACM
Successful delete: AEH_AGO-F
Successful delete: AEH_AER
Successful delete: AEH_ACG
Successful delete: AEG_AEE
Successful delete: AED_AHS
Successful delete: AF_AGD
Successful delete: AF_AFG
Successful delete: AF_ADC
Successful delete: AF_ACC
Successful delete: AEH_AGU
Successful delete: AED_AET
Successful delete: ABT_AGO-B
Successful delete: AGX_AGC
Successful delete: AF_ACE
Successful delete: AF_ABR
Successful delete: AES-C_AF
Successful delete: AEH_AGX
Successful delete: AEH_AGO
Successful delete: AEH_AEO
Successful delete: AEH_AEG
Successful delete: AEH_AED
Successful delete: AEH_ADM

In [364]:
test = list(cart[45][0][1])

In [745]:
s1 = list(cart[3][1][1])
s1.sort(key = lambda x: x[2])
s2 = [x[0] for x in s1]
s2
t1 = list(cart[3][0][1])
t1.sort(key = lambda x: x[2])
t2 = [x[0] for x in t1]
t2

[15.8,
 16.52,
 16.56,
 16.36,
 16.42,
 16.49,
 16.33,
 16.24,
 15.75,
 15.97,
 16.27,
 16.06,
 15.98,
 15.79,
 16.43,
 16.24,
 16.6,
 16.47,
 16.13,
 16.57,
 17.32,
 17.21,
 17.21,
 17.14,
 17.32,
 17.4,
 17.16,
 17.12,
 17.37,
 17.59,
 17.4,
 17.59,
 17.52,
 17.28,
 16.54,
 16.44,
 16.55,
 16.68,
 16.85,
 16.23,
 16.18,
 16.63,
 16.58,
 16.25,
 16.49,
 16.3,
 15.8,
 16.03,
 16.12,
 16.04,
 15.66,
 16.01,
 16.11,
 16.55,
 16.45,
 16.95,
 17.11,
 17.09,
 17.24,
 17.49,
 17.64,
 17.66,
 17.47,
 17.56,
 18.05,
 18.13,
 18.12,
 17.92,
 17.77,
 16.7,
 16.55,
 16.55,
 16.52,
 16.13,
 16.44,
 16.64,
 16.97,
 16.89,
 17.03,
 17.18,
 17.09,
 17.0,
 17.22,
 17.67,
 17.47,
 17.01,
 17.15,
 17.53,
 17.52,
 17.05,
 17.15,
 17.1,
 16.92,
 16.45,
 16.69,
 16.67,
 16.26,
 15.98,
 16.11,
 16.33,
 16.37,
 16.48,
 16.81,
 16.09,
 16.2,
 15.92,
 15.61,
 15.69,
 15.41,
 15.5,
 15.28,
 15.1,
 15.41,
 14.96,
 14.79,
 14.72,
 14.78,
 15.37,
 15.29,
 15.28,
 15.23,
 15.28,
 15.65,
 15.82,
 15.86,
 16.31,
 16.

In [878]:
import numpy as np
import statsmodels.api as stat
import statsmodels.tsa.stattools as ts

#return 1 if the two series are cointegrated and 0 otherwise

def eg_test(y, x):
    
    if len(y) == 0 | len(x) == 0:
        return [2,0,0]
    
    x = stat.add_constant(x)
    result = stat.OLS(y, x).fit()
    regPar = result.params
    adfResults = ts.adfuller(result.resid, maxlag=0, regression='c', autolag=None, store=False, regresults=True)
    tstat = adfResults[0]
    critVal = adfResults[2]['1%']
    
    if tstat < critVal:
        return [1,regPar]
    else:
        return [0,regPar]

In [901]:
eg_test(t2,s2)

[1, array([  1.26114397e-14,   1.00000000e+00])]

In [286]:
shared = sc.broadcast(d)

In [384]:
z = shared.value[0][1]

In [380]:
z.sort(key=lambda x: x[2])

In [667]:
myList = []
for i in range(0, 3000):
    for j in range(i+1,len(a)):
        myList.append(str(i) + '-' + str(j))
        
myList

['0-1',
 '0-2',
 '0-3',
 '0-4',
 '0-5',
 '0-6',
 '0-7',
 '0-8',
 '0-9',
 '1-2',
 '1-3',
 '1-4',
 '1-5',
 '1-6',
 '1-7',
 '1-8',
 '1-9',
 '2-3',
 '2-4',
 '2-5',
 '2-6',
 '2-7',
 '2-8',
 '2-9',
 '3-4',
 '3-5',
 '3-6',
 '3-7',
 '3-8',
 '3-9',
 '4-5',
 '4-6',
 '4-7',
 '4-8',
 '4-9',
 '5-6',
 '5-7',
 '5-8',
 '5-9',
 '6-7',
 '6-8',
 '6-9',
 '7-8',
 '7-9',
 '8-9']